# Data Cleaning

### Import Pacakages

In [3]:
import pandas as pd
import os
import re
import pandas as pd
import json
import os

### Create Unique ID for Participants

#### Prediction
Extract participant system-assigned IDs from trialdata.csv files across multiple movies, and assign a new standardized user ID (e.g., user_001, user_002, etc.) to each unique participant for use in downstream analysis.

In [6]:
movie_list = ["busstop", "cmiyc_long", "keithreynolds", "theboyfriend", "therock", "theshoe"]
base_path = "../../Raw Data/Prediction Raw Psiturk Data Files"
subfolder_list = ["Old_2019", "New_2021"]

base_ids_set = set()
full_ids_set = set()

for movie in movie_list:
    for subfolder in subfolder_list:
        trial_path = os.path.join(base_path, movie, subfolder, "trialdata.csv")
        print(f"\n📁 Reading data for: {movie} / {subfolder}")
        if not os.path.exists(trial_path):
            continue
        df = pd.read_csv(trial_path, header=None, comment="#")
        df.columns = ["ID", "Trial", "Timestamp", "Datastring"]
        print("  Participants in trial_data:", df["ID"].nunique())
        df["BaseID"] = df["ID"].astype(str).apply(lambda x: x.split(":")[0])
        base_ids_set.update(df["BaseID"].unique())
        full_ids_set.update(df["ID"].unique())

sorted_ids = sorted(base_ids_set)
user_id_map = {bid: f"{i+1:03d}" for i, bid in enumerate(sorted_ids)}


📁 Reading data for: busstop / Old_2019
  Participants in trial_data: 210

📁 Reading data for: busstop / New_2021
  Participants in trial_data: 85

📁 Reading data for: cmiyc_long / Old_2019
  Participants in trial_data: 260

📁 Reading data for: cmiyc_long / New_2021
  Participants in trial_data: 62

📁 Reading data for: keithreynolds / Old_2019
  Participants in trial_data: 198

📁 Reading data for: keithreynolds / New_2021
  Participants in trial_data: 64

📁 Reading data for: theboyfriend / Old_2019
  Participants in trial_data: 236

📁 Reading data for: theboyfriend / New_2021
  Participants in trial_data: 35

📁 Reading data for: therock / Old_2019
  Participants in trial_data: 204

📁 Reading data for: therock / New_2021
  Participants in trial_data: 77

📁 Reading data for: theshoe / Old_2019
  Participants in trial_data: 262

📁 Reading data for: theshoe / New_2021
  Participants in trial_data: 31


### Read Data

In [8]:
def read_data(base_path, movie, subfolder):
    """Read trial, event, and question data for a given movie and subfolder."""
    data_path = os.path.join(base_path, movie, subfolder)

    trial_path = os.path.join(data_path, "trialdata.csv")
    event_path = os.path.join(data_path, "eventdata.csv")
    question_path = os.path.join(data_path, "questiondata.csv")

    # Load trial data
    trial_data = pd.read_csv(trial_path, header=None, comment="#")
    trial_data.columns = ["ID", "Trial", "Timestamp", "Datastring"]

    # Load event data
    event_data = pd.read_csv(event_path, header=None, comment="#")
    event_data.columns = ["ID", "Event", "Duration", "Details", "Timestamp"]

    # Load question data
    question_data = pd.read_csv(question_path, header=None, comment="#")
    question_data.columns = ["ID", "Question", "Response"]

    return trial_data, event_data, question_data

### Drop Incomplete Subjects

In [10]:
def filter_completed_and_audio_catch(trial_data, event_data, question_data):
    """Keep only participants who appear in question_data and have completed Audio_Catch."""
    valid_ids = question_data["ID"].unique()

    audio_ids = question_data.loc[
        question_data["Question"] == "Audio_Catch", "ID"
    ].unique()

    keep_ids = set(valid_ids) & set(audio_ids)

    trial_data = trial_data[trial_data["ID"].isin(keep_ids)].copy()
    event_data = event_data[event_data["ID"].isin(keep_ids)].copy()
    question_data = question_data[question_data["ID"].isin(keep_ids)].copy()

    return trial_data, event_data, question_data

In [11]:
def filter_loops(trial_data, event_data, question_data, instr_threshold=2):
    """Remove participants who looped (repeated) experiments due to technical issue. 
       Filter by participants who clicked "begin" twice
    """
    def cnt(series):
        c = 0
        for ds in series.dropna():
            try:
                js = json.loads(ds)
            except (json.JSONDecodeError, TypeError):
                continue
            if str(js.get("phase")).lower() == "id_submit" and str(js.get("status")).lower() in {"begin", "submit"}:
                c += 1
        return c

    counts = trial_data.groupby("ID")["Datastring"].apply(cnt)

    drop_ids = set(counts[counts > 2].index.astype(str))
    
    EXEMPT_IDS = {"debug8CU6p:debugHfGpS", "debugGELZw:debugBfrPh"} # subject looped but not in middle of experiment phase
    
    drop_ids -= {str(x) for x in EXEMPT_IDS}

    trial_data = trial_data[~trial_data["ID"].isin(drop_ids)].copy()
    event_data = event_data[~event_data["ID"].isin(drop_ids)].copy()
    question_data = question_data[~question_data["ID"].isin(drop_ids)].copy()

    print(f"looped participants: {drop_ids}")
    
    return trial_data, event_data, question_data

In [12]:
def filter_trials(trial_data, question_data, correct_map):
    """Filter participants who passed comprehension test: all correct on 4 questions"""
    def phase(q):
        if isinstance(q, str):
            if "Comp_" in q:
                return "comprehension"
            if "Audio" in q:
                return "audio_catch"
        return None

    question_data["phase"] = question_data["Question"].apply(phase)

    comp = question_data[question_data["phase"]=="comprehension"].copy()
    comp["Response"] = pd.to_numeric(comp["Response"], errors="coerce")
    comp["acc"] = (comp["Response"] == comp["Question"].map(correct_map)).astype(int)

    audio = question_data[question_data["phase"]=="audio_catch"].copy()
    audio["acc"] = (audio["Response"] == audio["counterbalance"]).astype(int)

    drop_ids = set(audio.loc[audio["acc"]==0, "ID"]) | set(comp.loc[comp["acc"]==0, "ID"])

    return trial_data[~trial_data["ID"].isin(drop_ids)]


In [13]:
def filter_pilots(trial_data, question_data, event_data, THRESHOLD = 1555729200000):
    """Filter participants who start study after the study time. exclude pilot subjects"""

    trial_data["Timestamp"] = pd.to_numeric(trial_data["Timestamp"], errors="coerce")

    early_ids = trial_data.loc[trial_data["Timestamp"] < THRESHOLD, "OriginalID"] \
                          .dropna().unique().tolist()
    drop_pilots = list(set(early_ids))
    # print(drop_pilots)

    trial_data = trial_data[~trial_data["OriginalID"].isin(drop_pilots)]
    question_data = question_data[~question_data["ID"].isin(drop_pilots)]
    event_data = event_data[~event_data["ID"].isin(drop_pilots)]

    return trial_data, question_data, event_data


### Add New Unique ID

Anonymized participant IDs were generated using a standardized format that includes task type, year, and movie number (e.g., "P2023M12_001"). 

Base IDs were extracted from the original ID column, mapped to unique numbers, and combined with the corresponding prefix. 

The original ID values were preserved in a new column (`OriginalID`) for traceability. This process was applied consistently to all datasts.


In [16]:
def assign_new_user_ids(trial_data, event_data, question_data, user_id_map, task, year, movie):

    def map_ids(df):
        df = df.copy()
        df["BaseID"] = df["ID"].astype(str).apply(lambda x: x.split(":")[0])

        df["OriginalID"] = df["ID"] 

        df["UniqueNum"] = df["BaseID"].map(user_id_map)

        prefix = f"{task}{year}M{int(movie):02d}_"

        df["ID"] = df["UniqueNum"].apply(lambda x: f"{prefix}{str(x).zfill(3)}" if pd.notnull(x) else None)

        return df.drop(columns=["BaseID", "UniqueNum"])

    trial_data = map_ids(trial_data)
    event_data = map_ids(event_data)
    question_data = map_ids(question_data)

    return trial_data, event_data, question_data

### Extract Variables
Counterbalance

In [18]:
def add_counterbalance(trial_df, event_df, question_df):
    label_temp = trial_df[trial_df["Datastring"].str.contains("prac_segA_pred1", na=False)].copy()
    label_temp["BaseID"] = label_temp["ID"].astype(str).str.split(":").str[0]
    label_temp["counterbalance"] = label_temp["Datastring"].str.extract(r'"counterbalance"\s*:\s*([0-5])')[0]
    cb_map = dict(zip(label_temp["BaseID"], label_temp["counterbalance"]))

    def apply_map(df):
        df = df.copy()
        base = df["ID"].astype(str).str.split(":").str[0]
        df["counterbalance"] = base.map(cb_map)
        return df

    return apply_map(trial_df), apply_map(event_df), apply_map(question_df)

In [19]:
def filter_and_extract_tag(df, subfolder):
    df = df[df["Datastring"].str.contains(r'"phase"\s*:\s*"(?:prediction)"', na=False, regex=True)].copy()

    df["offset"] = df["Datastring"].str.extract(r'"offset"\s*:\s*"?(.*?)"?[,}]')
    df["tag"] = df["Datastring"].str.extract(r'"tag"\s*:\s*"?(.*?)"?[,}]')
    df["video_segment"] = df["tag"].str.extract(r'_seg([A-Za-z0-9]+)_')
    df["phase_type"] = df["tag"].apply(
        lambda x: "practice" if isinstance(x, str) and x.startswith("prac_")
        else ("test" if isinstance(x, str) and x.startswith("test_") else None)
    )

    df["data_set"] = subfolder
    return df


def extract_vidpath_prediction_time(df):
    """Extract vidpath, prediction number, time, and confidence from Datastring."""
    # vidpath
    df["vidpath"] = df["Datastring"].str.extract(r'"vidpath"\s*:\s*"([^"]+)"')

    # prediction number
    df["prediction_number"] = df["Datastring"].str.extract(r'"prediction"\s*:\s*"?(.*?)"?(?:,|})')[0]

    # time
    df["time"] = df["Datastring"].str.extract(r'"time"\s*:\s*([0-9]+)')

    # NEW: confidence
    df["confidence"] = df["Datastring"].str.extract(r'"confidence"\s*:\s*"?(.*?)"?(?:,|})')[0]

    for col in ["prediction_number", "time", "confidence", "offset"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    return df

def extract_prediction_content(df):
    df = df.copy()
    
    df["prediction_content"] = df["Datastring"].str.extract(r'"content"\s*:\s*"((?:\\.|[^"\\])*)"')

    df["prediction_content"] = df["prediction_content"].str.replace(r'\\"', r'\\', regex=True)

    return df


In [20]:
def reorder_and_remove_columns(df):
    desired_columns = [
        'ID', 'OriginalID', 'Trial', 'Timestamp', 'counterbalance', 'phase_type', 'confidence',
        'time', 'prediction_number', 'vidpath', 'prediction_content',
        'tag', 'offset', 'video_segment', 'data_set'
    ]
    
    # Only keep columns that exist in df
    cols_to_keep = [col for col in desired_columns if col in df.columns]
    return df[cols_to_keep]


## Run Data

### Prediction

In [23]:
def map_movie(movie):
    if movie == "busstop":
        return "01"
    elif movie == "cmiyc_long":
        return "02"
    elif movie == "keithreynolds":
        return "03"
    elif movie == "theboyfriend":
        return "04"
    elif movie == "therock":
        return "05"
    elif movie == "theshoe":
        return "06"
    else:
        return None
    
def map_year(year):
    if year == "Old_2019":
        return "2019"
    elif year == "New_2021":
        return "2021"
    else:
        return None

In [24]:
base_path = "../../Raw Data/Prediction Raw Psiturk Data Files"
movie_list = ["busstop", "cmiyc_long", "keithreynolds", "theboyfriend", "therock", "theshoe"]
subfolder_list = ["Old_2019", "New_2021"]

datasets_dic = {}
instr_threshold = 9
correct_map = {"Comp_Q1":1, "Comp_Q2":3, "Comp_Q3":4, "Comp_Q4":1}
THRESHOLD = 1555729200000


for movie in movie_list:
    print(movie)
    if movie == "theshoe":
        correct_map = {"Comp_Q1":2, "Comp_Q2":4, "Comp_Q3":1, "Comp_Q4":3}
        THRESHOLD = 1555027140000 # April 11, 2019
    else:
        correct_map = {"Comp_Q1":1, "Comp_Q2":3, "Comp_Q3":4, "Comp_Q4":1}
        THRESHOLD = 1555729200000 # April 19, 2019

    datasets = []
    for subfolder in subfolder_list:
        trial_data, event_data, question_data = read_data(base_path, movie, subfolder)
        
        trial_data, event_data, question_data = filter_completed_and_audio_catch(trial_data, event_data, question_data)
        trial_data, event_data, question_data = filter_loops(trial_data, event_data, question_data) 
        
        trial_data, event_data, question_data = add_counterbalance(trial_data, event_data, question_data)
        
        trial_data = filter_trials(trial_data, question_data, correct_map = correct_map)
        
        a = map_year(subfolder)
        b = map_movie(movie)
        trial_data, even_data, question_data = assign_new_user_ids(trial_data, event_data, question_data, user_id_map, 
                                                                   "P", a, b)
        trial_data, event_data, question_data = filter_pilots(trial_data, question_data, event_data, THRESHOLD)
        
        trial_data = filter_and_extract_tag(trial_data, subfolder)
        trial_data = extract_vidpath_prediction_time(trial_data)
        trial_data = extract_prediction_content(trial_data)
        trial_data = reorder_and_remove_columns(trial_data)

        print(subfolder+": data entries: "+str(len(trial_data)))
        
        datasets.append(trial_data)

    df_all = pd.concat(datasets, ignore_index=True)
    print(f"total number of rows:"+ str(len(df_all)))
    print("total number of participants: " + str(df_all["ID"].nunique()) + "\n")
    datasets_dic[movie] = df_all
    
    df_all.drop("OriginalID", axis = 1)

    filename = f"{movie}_prediction_cleaned.csv"
    output_dir = "../../Cleaned Data/Prediction_Cleaned_Data"
    os.makedirs(output_dir, exist_ok=True)

    output_path = os.path.join(output_dir, filename)
    df_all.to_csv(output_path, index=False)

busstop
looped participants: set()
Old_2019: data entries: 1753
looped participants: {'debugs2Uin:debugUss8N', 'debug617Hp:debugox7Nx', 'debugsgaz9:debugtsrSz', 'debugcWTZl:debugetWm9', 'debugE6AVE:debugdFR5k', 'debugBxi4k:debugiNh31', 'debugoO7XR:debugdSU0R'}
New_2021: data entries: 679
total number of rows:2432
total number of participants: 198

cmiyc_long
looped participants: set()
Old_2019: data entries: 2112
looped participants: {'debugNSQx8:debugypdvf', 'debugm3tNI:debug9KzKf'}
New_2021: data entries: 663
total number of rows:2775
total number of participants: 181

keithreynolds
looped participants: set()
Old_2019: data entries: 1289
looped participants: {'debugTD2AD:debugS2JLT', 'debugRl5wZ:debugBEuIb'}
New_2021: data entries: 438
total number of rows:1727
total number of participants: 184

theboyfriend
looped participants: set()
Old_2019: data entries: 1665
looped participants: set()
New_2021: data entries: 251
total number of rows:1916
total number of participants: 169

theroc

Data in each Movie:
`busstop`: 1753 + 679 = 2432
`cmiyc_long`: 2112 + 663 = 2775
`keithreynolds`: 1289 + 438 = 1727
`theboyfriend`: 1665 + 251 = 1916
`therock`: 1139 + 460 = 1599
`theshoe`: 960 + 117 = 1077

Removed subjects that looped: 
`busstop`: {'debugsgaz9:debugtsrSz', 'debugcWTZl:debugetWm9', 'debugBxi4k:debugiNh31', 'debugs2Uin:debugUss8N', 'debug617Hp:debugox7Nx', 'debugoO7XR:debugdSU0R', 'debugE6AVE:debugdFR5k'}
`cmiyc_long`: {'debugm3tNI:debug9KzKf', 'debugNSQx8:debugypdvf'}
`keithreynolds`: {'debugRl5wZ:debugBEuIb', 'debugTD2AD:debugS2JLT'}
`theshoe`: {'debugokHY2:debug7O47g'}

## Save ID map

In [47]:
combined = pd.concat(datasets_dic, names=["Movie"], ignore_index=False).reset_index(level=0).reset_index(drop=True)

user_id_map_df = combined[["ID", "OriginalID","Movie"]].drop_duplicates().reset_index(drop=True)

user_id_map_df.to_csv("../../Cleaned Data/Prediction_Cleaned_Data/prediction_user_id_map.csv", index=False)